[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RonPlusSign/AnomalySegmentation/blob/main/Project6.ipynb)

# **Anomaly Segmentation Project 6**
##*Andrea Delli, Christian Dellisanti, Giorgia Modi*

## Downloads

### Clone GitHub Repo and download validation datasets

In [ ]:
!pip  install -q numpy matplotlib Pillow torchvision visdom ood_metrics icecream cityscapesscripts tqdm

import sys, os
if not os.path.isfile('/content/Validation_Dataset.zip'):
  !gdown 12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta
if not os.path.isdir('/content/Validation_Dataset'):
  !unzip -q Validation_Dataset.zip
if not os.path.isdir('/content/AnomalySegmentation'):
  #!git clone https://github.com/shyam671/AnomalySegmentation_CourseProjectBaseCode.git
  !git clone -b main https://github.com/RonPlusSign/AnomalySegmentation.git

!cd /content/AnomalySegmentation && git pull
#!cd /content/AnomalySegmentation && git checkout main

#### Cityscape Dataset download

In [ ]:
import  os
# s306027@studenti.polito.it
# %mR+g$L\~5U03O9)IZ-_
createLabel = True
fast_download = True
super_fast_download = True
if super_fast_download:
  !gdown 1-fjLAk4_-GkixW1-GP_cYDBUmhnVbApL
  !unzip -q cityscapes.zip
  !mv  ./content/cityscapes /content/cityscapes
  !rm -rf ./content
else:
  if not os.path.isdir('/content/cityscapes'):
    !mkdir /content/cityscapes

  if not os.path.isfile('/content/cityscapes/gtFine_trainvaltest.zip'):
    if fast_download:
      !gdown 1J31rnVd33GBt-IYGYqC9mv73q7vc55pw -O /content/cityscapes/
    else:
      !csDownload gtFine_trainvaltest.zip -d /content/cityscapes/

  if not os.path.isdir('/content/cityscapes/gtFine'):
    !unzip -q /content/cityscapes/gtFine_trainvaltest.zip -d /content/cityscapes/
    createLabel = True
    !rm /content/cityscapes/README
    !rm /content/cityscapes/license.txt

  if not os.path.isfile('/content/cityscapes/leftImg8bit_trainvaltest.zip'):
    if fast_download:
      #https://drive.google.com/file/d/1m8Y3Zc6vG11Q9SxW7Be5EGXTDq4s4RlJ/view?usp=sharing
      !gdown 1m8Y3Zc6vG11Q9SxW7Be5EGXTDq4s4RlJ -O /content/cityscapes/
    else:
      !csDownload leftImg8bit_trainvaltest.zip -d /content/cityscapes/

  if not os.path.isdir('/content/cityscapes/leftImg8bit'):
    !unzip -q /content/cityscapes/leftImg8bit_trainvaltest.zip -d /content/cityscapes/
    createLabel = True
    !rm /content/cityscapes/README
    !rm /content/cityscapes/license.txt

  if createLabel:
    os.environ['CITYSCAPES_DATASET'] = '/content/cityscapes/'
    !csCreateTrainIdLabelImgs

Downloading...
From (original): https://drive.google.com/uc?id=1-fjLAk4_-GkixW1-GP_cYDBUmhnVbApL
From (redirected): https://drive.google.com/uc?id=1-fjLAk4_-GkixW1-GP_cYDBUmhnVbApL&confirm=t&uuid=23fcbdef-dadc-4978-ae8b-b40816dd49aa
To: /content/cityscapes.zip
100% 11.9G/11.9G [03:02<00:00, 65.4MB/s]


##Baseline

### Evaluation

**ERFet anomaly Inference with MSP, MaxLogit, MaxEntropy and Mahalanobis methods.**

In [ ]:
import torch

no_execute = False
just_once = False

for dataset_dir in ['RoadAnomaly21', 'RoadObsticle21', 'FS_LostFound_full', 'fs_static', 'RoadAnomaly']:
  print("----------------------------")
  for method in ["MSP", "MaxLogit", "MaxEntropy"]:

    if no_execute:
      break

    format_file = os.listdir(f'/content/Validation_Dataset/{dataset_dir}/images')[0].split(".")[1]
    input =f'/content/Validation_Dataset/{dataset_dir}/images/\*.{format_file}'
    print(f"\nDataset: {dataset_dir} method: {method}")
    save_plots_dir = f'/content/AnomalySegmentation/plots/{dataset_dir}_{method}'

    if torch.cuda.is_available():
      !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method {method} --save-plots-dir {save_plots_dir} | tail -n 2
    else:
      !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method {method} --save-plots-dir {save_plots_dir} --cpu | tail -n 2

    if just_once:
      no_execute = True
      just_once = False

----------------------------

Dataset: RoadAnomaly21 method: MSP
AUPRC score: 29.100168300581203
FPR@TPR95: 62.51075321069286

Dataset: RoadAnomaly21 method: MaxLogit
AUPRC score: 38.31957797222208
FPR@TPR95: 59.3370558914899

Dataset: RoadAnomaly21 method: MaxEntropy
AUPRC score: 31.005102648344756
FPR@TPR95: 62.593151130093226
----------------------------

Dataset: RoadObsticle21 method: MSP
AUPRC score: 2.7116243119338366
FPR@TPR95: 64.9739786894368

Dataset: RoadObsticle21 method: MaxLogit
AUPRC score: 4.626567617520253
FPR@TPR95: 48.443439151949555

Dataset: RoadObsticle21 method: MaxEntropy
AUPRC score: 3.051560023478638
FPR@TPR95: 65.59968252759046
----------------------------

Dataset: FS_LostFound_full method: MSP
AUPRC score: 1.747872547607269
FPR@TPR95: 50.76348570192957

Dataset: FS_LostFound_full method: MaxLogit
AUPRC score: 3.3014401015087245
FPR@TPR95: 45.494876929038305

Dataset: FS_LostFound_full method: MaxEntropy
AUPRC score: 2.581709137723009
FPR@TPR95: 50.36809978

###Baseline mIoU

In [ ]:
import torch

if torch.cuda.is_available():
  !python -W ignore /content/AnomalySegmentation/eval/eval_iou.py --loadDir /content/AnomalySegmentation/trained_models/ --datadir /content/cityscapes/  | tail -n 28
else:
  !python -W ignore /content/AnomalySegmentation/eval/eval_iou.py  --loadDir  /content/AnomalySegmentation/trained_models/ --datadir /content/cityscapes/  --cpu | tail -n 28

498 val/munster/munster_000172_000019_leftImg8bit.png
499 val/munster/munster_000173_000019_leftImg8bit.png
-------------MSP-------------------
---------------------------------------
Took  80.77754092216492 seconds
Per-Class IoU:
97.62 Road
81.37 sidewalk
90.77 building
49.43 wall
54.93 fence
60.81 pole
62.60 traffic light
72.32 traffic sign
91.35 vegetation
60.97 terrain
93.38 sky
76.11 person
53.45 rider
92.91 car
72.78 truck
78.87 bus
63.86 train
46.41 motorcycle
71.89 bicycle
MEAN IoU:  72.20 %


##Temperature Scaling

### Evaluation

**ERFNet anomaly inference with MSP with temperature scaling.**

In [ ]:
import torch

no_execute = False
just_once = False
for dataset_dir in ['RoadAnomaly21', 'RoadObsticle21', 'FS_LostFound_full', 'fs_static', 'RoadAnomaly']:
  print("----------------------------")
  for t in [0.5, 0.75, 1.1]:
    if no_execute:
        break

    format_file = os.listdir(f'/content/Validation_Dataset/{dataset_dir}/images')[0].split(".")[1]
    input =f'/content/Validation_Dataset/{dataset_dir}/images/\*.{format_file}'
    print(f"\nDataset: {dataset_dir}, method: MSP, Temperature: {t}")
    if torch.cuda.is_available():
      !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method 'MSP' --temperature {t} | tail -n 2
    else:
      !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method 'MSP' --cpu --temperature {t} | tail -n 2

    if just_once:
      no_execute = True
      just_once = False

----------------------------

Dataset: RoadAnomaly21 method : MSP Temperature: 0.5
AUPRC score: 27.060833635879618
FPR@TPR95: 62.730810427606734

Dataset: RoadAnomaly21 method : MSP Temperature: 0.75
AUPRC score: 28.156063054348103
FPR@TPR95: 62.478737323984326

Dataset: RoadAnomaly21 method : MSP Temperature: 1.1
AUPRC score: 29.40955379121979
FPR@TPR95: 62.58986549662704
----------------------------

Dataset: RoadObsticle21 method : MSP Temperature: 0.5
AUPRC score: 2.4195519558429823
FPR@TPR95: 63.22544524787239

Dataset: RoadObsticle21 method : MSP Temperature: 0.75
AUPRC score: 2.5668802249367677
FPR@TPR95: 64.05285534718263

Dataset: RoadObsticle21 method : MSP Temperature: 1.1
AUPRC score: 2.7658075767433776
FPR@TPR95: 65.52358106228223
----------------------------

Dataset: FS_LostFound_full method : MSP Temperature: 0.5
AUPRC score: 1.2802500246431052
FPR@TPR95: 66.73710676943257

Dataset: FS_LostFound_full method : MSP Temperature: 0.75
AUPRC score: 1.4927065686510383
FPR@TPR

## Void Classifier

### Fine-tuning

**Fine-tune ERFNet, ENet and BiSeNetv1 for VOID classification using Cross Entropy loss.**

In [ ]:
# Fine-tune ERFNet, ENet and BiSeNetv1 for VOID classification using CrossEntropy

models = ["erfnet", "enet", "bisenet"]
savedirs = ["erfnet_training_void", "enet_training_void", "bisenet_training_void"]
pretrained_weights = ["erfnet_pretrained.pth", "enet_pretrained.pth", "bisenetv1_pretrained.pth"]
epochs = 20

# Base directory of the project
base_dir = "/content/AnomalySegmentation/train"
# Dataset directory
data_dir = "/content/cityscapes"

# Loop to execute fine-tuning
for model, savedir, pretrained_weight  in zip(models, savedirs, pretrained_weights):
    print(f"\n\n----- Fine-tuning {model} for VOID classification -----")
    !cd {base_dir} && python -W ignore main.py --savedir {savedir} --datadir {data_dir} --model {model} --cuda --num-epochs=20 --epochs-save=1 --FineTune --decoder --loadWeights={pretrained_weight}
    print(f"Model saved in /content/AnomalySegmentation/save/{savedir}")
    # zip folder
    !zip -r save_{savedir}.zip /content/AnomalySegmentation/save/{savedir}



----- Fine-tuning erfnet for VOID classification -----
Import Model erfnet with weights erfnet_pretrained.pth to FineTune
========== TRAINING ===========
/content/cityscapes/leftImg8bit/train
/content/cityscapes/leftImg8bit/val
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  5e-05
loss: 0.326 (epoch: 1, step: 0) // Avg time/img: 0.4121 s
loss: 0.4037 (epoch: 1, step: 50) // Avg time/img: 0.0468 s
loss: 0.3878 (epoch: 1, step: 100) // Avg time/img: 0.0436 s
loss: 0.3854 (epoch: 1, step: 150) // Avg time/img: 0.0425 s
loss: 0.3806 (epoch: 1, step: 200) // Avg time/img: 0.0422 s
loss: 0.3821 (epoch: 1, step: 250) // Avg time/img: 0.0420 s
loss: 0.3818 (epoch: 1, step: 300) // Avg time/img: 0.0421 s
loss: 0.3768 (epoch: 1, step: 350) // Avg time/img: 0.0421 s
loss: 0.3736 (epoch: 1, step: 400) // Avg time/img: 0.0420 s
loss: 0.3724 (epoch: 1, step: 450) // Avg time/img: 0.0420 s
----- VALIDATING - EPOCH 1 -----
VAL loss: 0.4843 (epoch: 1, step: 0) // Avg time/img: 0.0493 s
VAL loss: 0.577

###Evaluation

In [ ]:
import torch

no_execute = False
just_once = False

for net in ["erfnet", "enet", "bisenet"]:
  print("----------------------------")
  for dataset_dir in ['RoadAnomaly21', 'RoadObsticle21', 'FS_LostFound_full', 'fs_static', 'RoadAnomaly']:

    if no_execute:
      break

    load_dir = f'/content/AnomalySegmentation/save/{net}_training_void'
    weights = f'/model_best.pth'
    format_file = os.listdir(f'/content/Validation_Dataset/{dataset_dir}/images')[0].split(".")[1]
    input =f'/content/Validation_Dataset/{dataset_dir}/images/\*.{format_file}'
    print(f"\nDataset: {dataset_dir} net: {net}")

    if torch.cuda.is_available():
      !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} | tail -n 2
    else:
      !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} --cpu | tail -n 2

    if just_once:
      no_execute = True
      just_once = False

----------------------------

Dataset: RoadAnomaly21 net: erfnet
100% 10/10 [00:01<00:00,  5.05it/s]
AUPRC score: 20.96027450905394
FPR@TPR95: 70.65199818240566

Dataset: RoadObsticle21 net: erfnet
100% 30/30 [00:06<00:00,  4.74it/s]
AUPRC score: 0.9534509464686635
FPR@TPR95: 99.70382904516488

Dataset: FS_LostFound_full net: erfnet
100% 100/100 [00:22<00:00,  4.52it/s]
AUPRC score: 11.954664299544238
FPR@TPR95: 13.173385116458128

Dataset: fs_static net: erfnet
100% 30/30 [00:03<00:00,  7.69it/s]
AUPRC score: 19.089759422447592
FPR@TPR95: 54.49275015324435

Dataset: RoadAnomaly net: erfnet
100% 60/60 [00:07<00:00,  8.30it/s]
AUPRC score: 9.616979144835998
FPR@TPR95: 89.60496907078064
----------------------------


Dataset: RoadAnomaly21 net: enet
100% 10/10 [00:02<00:00,  4.72it/s]
AUPRC score: 12.821526992462903
FPR@TPR95: 96.94685438578138

Dataset: RoadObsticle21 net: enet
100% 30/30 [00:06<00:00,  4.68it/s]
AUPRC score: 0.6694272072184433
FPR@TPR95: 99.8967014003895

Dataset: FS_L

###mIoU Void Classification

In [ ]:
import torch
no_execute = False
just_once = False

for model in ["erfnet", "enet", "bisenet"]:

  if no_execute:
      break
  print(f"-----------{model}-------------")
  loadDir = f'/content/AnomalySegmentation/save/{model}_training_void'
  weights = f'/model_best.pth'
  if torch.cuda.is_available():
    !python -W ignore /content/AnomalySegmentation/eval/eval_iou.py --loadDir {loadDir} --loadWeights {weights} --datadir /content/cityscapes/ --model {model} | tail -n 25
  else:
    !python -W ignore /content/AnomalySegmentation/eval/eval_iou.py  --loadDir {loadDir} --loadWeights {weights} --datadir /content/cityscapes/  --model {model}  --cpu | tail -n 25


  if just_once:
    no_execute = True
    just_once = False

-----------erfnet-------------
---------------------------------------
Took  76.07235813140869 seconds
Per-Class IoU:
97.25 Road
81.05 sidewalk
90.53 building
49.73 wall
54.85 fence
60.75 pole
62.61 traffic light
72.16 traffic sign
91.30 vegetation
60.59 terrain
93.18 sky
74.87 person
53.61 rider
92.58 car
72.94 truck
78.58 bus
65.38 train
44.76 motorcycle
70.18 bicycle
MEAN IoU:  71.94 %
-----------enet-------------
---------------------------------------
Took  78.80453968048096 seconds
Per-Class IoU:
71.58 Road
26.36 sidewalk
70.53 building
4.72 wall
26.07 fence
36.33 pole
18.47 traffic light
35.14 traffic sign
80.58 vegetation
27.30 terrain
82.60 sky
19.26 person
6.85 rider
75.54 car
5.68 truck
12.02 bus
10.80 train
0.01 motorcycle
45.33 bicycle
MEAN IoU:  34.48 %
-----------bisenet-------------
---------------------------------------
Took  76.26429104804993 seconds
Per-Class IoU:
96.96 Road
76.78 sidewalk
89.30 building
50.16 wall
43.37 fence
47.45 pole
53.08 traffic light
64.41 tr

# Extensions

##Mahalanobis

Compute Cityscape mean vectors and covariance matrix for 19 classes.

In [ ]:
import numpy as np
datadir = '/content/cityscapes'
loadWeights = '/trained_models/erfnet_pretrained.pth'
loadDir = '/content/AnomalySegmentation'

# Compute Dataset Mean
!python /content/AnomalySegmentation/eval/mahalanobis.py --datadir {datadir} --loadDir {loadDir} --loadWeights {loadWeights}

# Compute Dataset Covariance
mean = "/save/mean_cityscapes_erfnet.npy"
!python /content/AnomalySegmentation/eval/mahalanobis.py --datadir {datadir} --loadDir {loadDir} --loadWeights {loadWeights} --mean {mean} --num-workers 2

Loading model: erfnet
Loading weights: /content/AnomalySegmentation/trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
100% 2975/2975 [07:55<00:00,  6.26it/s]
Mean output saved as '/content/AnomalySegmentation/save/mean_cityscapes_erfnet.npy'
Loading model: erfnet
Loading weights: /content/AnomalySegmentation/trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
100% 2975/2975 [07:41<00:00,  6.45it/s]
Covariance matrix: torch.Size([19, 19])
Covariance matrice saved as '/content/AnomalySegmentation/save/cov_matrix_erfnet.npy'


In [ ]:
import torch

no_execute = False
just_once = False

for dataset_dir in ['RoadAnomaly21', 'RoadObsticle21', 'FS_LostFound_full', 'fs_static', 'RoadAnomaly']:
  print("----------------------------")
  for method in ["Mahalanobis"]: # ["Mahalanobis", "MSP", "MaxLogit", "MaxEntropy"]:

    if no_execute:
      break

    format_file = os.listdir(f'/content/Validation_Dataset/{dataset_dir}/images')[0].split(".")[1]
    input =f'/content/Validation_Dataset/{dataset_dir}/images/\*.{format_file}'
    print(f"\nDataset: {dataset_dir} method: {method}")

    if torch.cuda.is_available():
      !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method  {method} | tail -n 2
    else:
      !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method {method}  --cpu | tail -n 2

    if just_once:
      no_execute = True
      just_once = False

----------------------------

Dataset: RoadAnomaly21 method: Mahalanobis
100% 10/10 [00:02<00:00,  3.75it/s]
AUPRC score: 30.882157230765973
FPR@TPR95: 74.4879201650216
----------------------------

Dataset: RoadObsticle21 method: Mahalanobis
100% 30/30 [00:08<00:00,  3.63it/s]
AUPRC score: 9.641072388098362
FPR@TPR95: 52.43062225917142
----------------------------

Dataset: FS_LostFound_full method: Mahalanobis
100% 100/100 [00:26<00:00,  3.78it/s]
AUPRC score: 2.9424493821312754
FPR@TPR95: 55.231371209083505
----------------------------

Dataset: fs_static method: Mahalanobis
100% 30/30 [00:06<00:00,  4.96it/s]
AUPRC score: 8.926899026567218
FPR@TPR95: 39.33961912182676
----------------------------

Dataset: RoadAnomaly method: Mahalanobis
100% 60/60 [00:11<00:00,  5.39it/s]
AUPRC score: 13.528992306695075
FPR@TPR95: 79.6254790023211


## Losses

### Fine-tuning

In [ ]:
# Fine tune ERFNET with different losses
"""
Training:
1. Focal loss
2. LogitNorm+CrossEntropy loss
3. IsoMaxPlus+CrossEntropy loss
4. LogitNorm+Focal loss
5. IsoMaxPlus+Focal loss
"""
titles = ["Focal", "LogitNorm+CrossEntropy", "IsoMaxPlus+CrossEntropy", "LogitNorm+Focal", "IsoMaxPlus+Focal"]
losses = ["Focal", "CrossEntropy", "CrossEntropy", "Focal", "Focal"]
models = ["erfnet", "erfnet", "erfnet_isomaxplus", "erfnet", "erfnet_isomaxplus"]
savedirs = ["erfnet_training_focal_loss", "erfnet_training_logitnorm_cross_entropy_loss", "erfnet_training_isomaxplus_cross_entropy_loss", "erfnet_training_logitnorm_focal_loss", "erfnet_training_isomaxplus_focal_loss"]
logit_normalization_flags = [False, True, False, True, False]
epochs = 20

# Base directory of the project
base_dir = "/content/AnomalySegmentation/train"
# Dataset directory
data_dir = "/content/cityscapes"
pretrained_weights = "erfnet_pretrained.pth"

# Loop to execute fine-tuning
for title, loss, model, savedir, logit_normalization_flag in zip(titles, losses, models, savedirs, logit_normalization_flags):
    print(f"\n\n----- Fine-tuning with {title} loss -----")
    !cd {base_dir} && python -W ignore main.py --savedir {savedir} --loss {loss} --logit_normalization {logit_normalization_flag} --datadir {data_dir} --model {model} --cuda --num-epochs=20 --epochs-save=1 --FineTune --decoder --loadWeights={pretrained_weights}
    print(f"Model saved in /content/AnomalySegmentation/save/{savedir}")
    # zip folder
    !zip -r save_{savedir}.zip /content/AnomalySegmentation/save/{savedir}



----- Fine-tuning with Focal loss -----
Import Model erfnet with weights erfnet_pretrained.pth to FineTune
========== TRAINING ===========
/content/cityscapes/leftImg8bit/train
/content/cityscapes/leftImg8bit/val
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  5e-05
loss: 0.8898 (epoch: 1, step: 0) // Avg time/img: 0.3255 s
loss: 1.234 (epoch: 1, step: 50) // Avg time/img: 0.0495 s
loss: 1.208 (epoch: 1, step: 100) // Avg time/img: 0.0470 s
loss: 1.209 (epoch: 1, step: 150) // Avg time/img: 0.0462 s
loss: 1.19 (epoch: 1, step: 200) // Avg time/img: 0.0459 s
loss: 1.188 (epoch: 1, step: 250) // Avg time/img: 0.0458 s
loss: 1.185 (epoch: 1, step: 300) // Avg time/img: 0.0456 s
loss: 1.169 (epoch: 1, step: 350) // Avg time/img: 0.0456 s
loss: 1.169 (epoch: 1, step: 400) // Avg time/img: 0.0455 s
loss: 1.16 (epoch: 1, step: 450) // Avg time/img: 0.0455 s
----- VALIDATING - EPOCH 1 -----
VAL loss: 1.152 (epoch: 1, step: 0) // Avg time/img: 0.0486 s
VAL loss: 1.342 (epoch: 1, step: 50) // A

###Evaluation

In [ ]:
import torch

no_execute = False
just_once = False

losses = ["CrossEntropy", "Focal", "LogitNorm+CrossEntropy", "IsoMaxPlus+CrossEntropy", "LogitNorm+FocalLoss", "IsoMaxPlus+FocalLoss"]
models = ["erfnet", "erfnet", "erfnet", "erfnet_isomaxplus", "erfnet", "erfnet_isomaxplus"]
load_dirs = ["/content/AnomalySegmentation/trained_models/", "/content/AnomalySegmentation/save/erfnet_training_focal_loss/", "/content/AnomalySegmentation/save/erfnet_training_logitnorm_cross_entropy_loss/", "/content/AnomalySegmentation/save/erfnet_training_isomaxplus_cross_entropy_loss/", "/content/AnomalySegmentation/save/erfnet_training_logitnorm_focal_loss/", "/content/AnomalySegmentation/save/erfnet_training_isomaxplus_focal_loss/"]
weights = ["erfnet_pretrained.pth", "model_best.pth", "model_best.pth", "model_best.pth", "model_best.pth", "model_best.pth"]

for loss, model, load_dir, weight in zip(losses, models, load_dirs, weights):
  print(f"------ Evaluating loss: {loss} ------\n")
  for dataset_dir in ['RoadAnomaly21', 'RoadObsticle21', 'FS_LostFound_full', 'fs_static', 'RoadAnomaly']:
    for method in ["MSP", "MaxLogit", "MaxEntropy", "Mahalanobis"]:

      if no_execute:
        break

      format_file = os.listdir(f'/content/Validation_Dataset/{dataset_dir}/images')[0].split(".")[1]
      input =f'/content/Validation_Dataset/{dataset_dir}/images/\*.{format_file}'

      print(f"\nDataset: {dataset_dir} method: {method} loss: {loss}")

      if torch.cuda.is_available():
        !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method  {method} --model {model} --loadDir {load_dir} --loadWeights {weight} | tail -n 2
      else:
        !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method {method}  --model {model} --loadDir {load_dir} --loadWeights {weight} --cpu | tail -n 2

      print("----------------------------")
      if just_once:
        no_execute = True
        just_once = False
    print("----------------------------\n\n")

------ Evaluating loss: CrossEntropy ------


Dataset: RoadAnomaly21 method: MSP loss: CrossEntropy
100% 10/10 [00:03<00:00,  2.77it/s]
AUPRC score: 29.100168300581203
FPR@TPR95: 62.51075321069286
----------------------------

Dataset: RoadAnomaly21 method: MaxLogit loss: CrossEntropy
100% 10/10 [00:02<00:00,  4.28it/s]
AUPRC score: 38.31957797222208
FPR@TPR95: 59.3370558914899
----------------------------

Dataset: RoadAnomaly21 method: MaxEntropy loss: CrossEntropy
100% 10/10 [00:02<00:00,  4.53it/s]
AUPRC score: 31.005102648344756
FPR@TPR95: 62.593151130093226
----------------------------

Dataset: RoadAnomaly21 method: Mahalanobis loss: CrossEntropy
100% 10/10 [00:02<00:00,  3.72it/s]
AUPRC score: 30.882157230765973
FPR@TPR95: 74.4879201650216
----------------------------
----------------------------



Dataset: RoadObsticle21 method: MSP loss: CrossEntropy
100% 30/30 [00:07<00:00,  4.25it/s]
AUPRC score: 2.7116243119338366
FPR@TPR95: 64.9739786894368
----------------------------



### mIoU Losses

In [ ]:
import torch
no_execute = False
just_once = False

for loss in ["crossentropy", "focal", "logitnorm", "isomaxplus"]:

  if no_execute:
      break
  print(f"-----------erfnet_{loss}_loss-------------")

  if loss == "crossentropy":
    loadDir = '/content/AnomalySegmentation/trained_models'
    weights = '/erfnet_pretrained.pth'
  else:
    loadDir = f'/content/AnomalySegmentation/save/erfnet_training_{loss}_loss'
    weights = f'/model_best.pth'

  if loss == "isomaxplus":
    model = "erfnet_isomaxplus"
  else:
    model = "erfnet"

  if torch.cuda.is_available():
    !python -W ignore /content/AnomalySegmentation/eval/eval_iou.py --loadDir {loadDir} --loadWeights {weights} --datadir /content/cityscapes/ --model {model} | tail -n 25
  else:
    !python -W ignore /content/AnomalySegmentation/eval/eval_iou.py  --loadDir {loadDir} --loadWeights {weights} --datadir /content/cityscapes/  --model {model}  --cpu | tail -n 25


  if just_once:
    no_execute = True
    just_once = False

-----------erfnet_crossentropy_loss-------------
---------------------------------------
Took  76.16176652908325 seconds
Per-Class IoU:
97.62 Road
81.37 sidewalk
90.77 building
49.43 wall
54.93 fence
60.81 pole
62.60 traffic light
72.32 traffic sign
91.35 vegetation
60.97 terrain
93.38 sky
76.11 person
53.45 rider
92.91 car
72.78 truck
78.87 bus
63.86 train
46.41 motorcycle
71.89 bicycle
MEAN IoU:  72.20 %
-----------erfnet_focal_loss-------------
---------------------------------------
Took  77.0158748626709 seconds
Per-Class IoU:
97.57 Road
81.37 sidewalk
90.87 building
49.48 wall
54.50 fence
61.06 pole
63.19 traffic light
72.69 traffic sign
91.45 vegetation
61.25 terrain
93.53 sky
76.52 person
53.82 rider
92.84 car
71.65 truck
78.42 bus
64.67 train
45.35 motorcycle
71.50 bicycle
MEAN IoU:  72.20 %
-----------erfnet_logitnorm_loss-------------
---------------------------------------
Took  75.2553186416626 seconds
Per-Class IoU:
97.67 Road
81.65 sidewalk
90.76 building
49.53 wall
54.5

# Visualization

In [ ]:
import torch

# Example image to color
# Nice images: RoadAnomaly/images/28, RoadAnomaly/images/58
input = '/content/Validation_Dataset/RoadAnomaly/images/58.jpg'

### Baseline models ###
for method in ["MSP", "MaxLogit", "MaxEntropy", "Mahalanobis"]:
  print(f"Method: {method}")
  save_image_path = f'/content/AnomalySegmentation/visualization/baseline/{method}'

  if torch.cuda.is_available():
    !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method  {method} --save-colored {save_image_path}  | tail -n 2
  else:
    !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method {method} --save-colored {save_image_path} --cpu | tail -n 2

!python /content/AnomalySegmentation/eval/visualization.py --name_dir="/content/AnomalySegmentation/visualization/baseline" --name_output="/content/AnomalySegmentation/visualization/baseline_visualization.png"

### Temperature scaling ###
for t in [0.5, 0.75, 1.1]:
  print(f"Method: MSP, Temperature: {t}")
  save_image_path = f'/content/AnomalySegmentation/visualization/temperature/t={t}'

  if torch.cuda.is_available():
    !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method 'MSP' --temperature {t} --save-colored {save_image_path} | tail -n 2
  else:
    !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method 'MSP' --cpu --temperature {t} --save-colored {save_image_path} | tail -n 2

!python /content/AnomalySegmentation/eval/visualization.py --name_dir="/content/AnomalySegmentation/visualization/temperature" --name_output="/content/AnomalySegmentation/visualization/temperature_visualization.png"

### Finetuned models with void ###
for net in ["erfnet", "enet", "bisenet"]:
  save_image_path = f'/content/AnomalySegmentation/visualization/void/{net}'
  load_dir = f'/content/AnomalySegmentation/save/{net}_training_void'
  weights = f'/model_best.pth'
  print(f"Finetuned network: {net}")
  if torch.cuda.is_available():
    !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} --save-colored {save_image_path} | tail -n 2
  else:
    !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} --cpu --save-colored {save_image_path} | tail -n 2

!python /content/AnomalySegmentation/eval/visualization.py --name_dir="/content/AnomalySegmentation/visualization/void" --name_output="/content/AnomalySegmentation/visualization/void_visualization.png"

### Losses ###
losses = ["CrossEntropy", "Focal", "LogitNorm", "IsoMaxPlus"]
models = ["erfnet", "erfnet", "erfnet", "erfnet_isomaxplus"]
load_dirs = ["/content/AnomalySegmentation/trained_models/", "/content/AnomalySegmentation/save/erfnet_training_focal_loss/", "/content/AnomalySegmentation/save/erfnet_training_logitnorm_loss/", "/content/AnomalySegmentation/save/erfnet_training_isomaxplus_loss/"]
weights = ["erfnet_pretrained.pth", "model_best.pth", "model_best.pth", "model_best.pth"]
for loss, model, load_dir, weight in zip(losses, models, load_dirs, weights):
  for method in ["MSP", "MaxLogit", "MaxEntropy", "Mahalanobis"]:
    save_image_path = f'/content/AnomalySegmentation/visualization/losses/{loss}/{method}'
    print(f"Method: {method}, loss: {loss}")
    if torch.cuda.is_available():
      !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method  {method} --model {model} --loadDir {load_dir} --loadWeights {weight} --save-colored {save_image_path} | tail -n 2
    else:
      !python -W ignore /content/AnomalySegmentation/eval/evalAnomaly.py --input {input} --method {method}  --model {model} --loadDir {load_dir} --loadWeights {weight} --save-colored {save_image_path} --cpu | tail -n 2

!python /content/AnomalySegmentation/eval/visualization.py --name_dir="/content/AnomalySegmentation/visualization/losses/CrossEntropy" --name_output="/content/AnomalySegmentation/visualization/losses_CrossEntropy_visualization.png"
!python /content/AnomalySegmentation/eval/visualization.py --name_dir="/content/AnomalySegmentation/visualization/losses/Focal" --name_output="/content/AnomalySegmentation/visualization/losses_Focal_visualization.png"
!python /content/AnomalySegmentation/eval/visualization.py --name_dir="/content/AnomalySegmentation/visualization/losses/LogitNorm" --name_output="/content/AnomalySegmentation/visualization/losses_LogitNorm_visualization.png"
!python /content/AnomalySegmentation/eval/visualization.py --name_dir="/content/AnomalySegmentation/visualization/losses/IsoMaxPlus" --name_output="/content/AnomalySegmentation/visualization/losses_IsoMaxPlus_visualization.png"

# Zip the images
!zip -r colored_anomalies.zip /content/AnomalySegmentation/visualization

Method: MSP
100% 1/1 [00:01<00:00,  1.19s/it]
AUPRC score: 45.49713738080089
FPR@TPR95: 33.597166197756
Method: MaxLogit
100% 1/1 [00:01<00:00,  1.17s/it]
AUPRC score: 70.57981429550465
FPR@TPR95: 24.507008237008915
Method: MaxEntropy
100% 1/1 [00:01<00:00,  1.27s/it]
AUPRC score: 57.507333629583414
FPR@TPR95: 33.14569845089997
Method: Mahalanobis
100% 1/1 [00:01<00:00,  1.25s/it]
AUPRC score: 39.617639754156656
FPR@TPR95: 26.45312870750144
Image saved as /content/AnomalySegmentation/visualization/baseline_visualization.png
Method: MSP, Temperature: 0.5
100% 1/1 [00:01<00:00,  1.21s/it]
AUPRC score: 37.35836052078424
FPR@TPR95: 34.936574654719806
Method: MSP, Temperature: 0.75
100% 1/1 [00:01<00:00,  1.52s/it]
AUPRC score: 40.98644320732818
FPR@TPR95: 34.28464289346124
Method: MSP, Temperature: 1.1
100% 1/1 [00:01<00:00,  1.26s/it]
AUPRC score: 47.3224440740321
FPR@TPR95: 33.33954781193858
Image saved as /content/AnomalySegmentation/visualization/temperature_visualization.png
Finetuned